#### 1. 查看系统平均负载
```shell
➜  uptime
11:20  up 3 days, 18:14, 3 users, load averages: 1.87 1.81 1.75
```
这几列分别是: 
* 系统当前时间
* 系统已启动的时间
* 当前用登陆的用户数
* 近1分钟,5分钟,15分钟的平均负载

#### 2. 什么是平均负载
平均负载是指: 单位时间内, 处于可运行态和不可中断态的进程数量
* 可运行态: 正在使用CPU或等待CPU时间片的进程数量
* 不可中断态: 这些进程不可被打断(interrupt), 指在内核态关键流程中的进行. 比如等待硬件设备IO的进程

#### 3. 平均负载与CPU使用率的区别
* CPU使用率代表CPU核数使用的比率;   
 而平均负载不仅包括正在使用CPU的进程数量, 还包括等待CPU和等待I/O的进程的数量
* CPU使用率和平均负载不一定完全成正比 : 
    * CPU密集型进程:  
     使用大量CPU,导致平均负载升高, 此时呈正比
    * IO密集型进程:  
     大量进程等待硬件IO, CPU使用率不高. 但是平均负载很高
    * 大量等待CPU进程调度也会导致平均负载升高, 此时的CPU使用率也会比较高/
    

#### 4. 使用stress与sysstat模拟演示
* 安装`stress`与`sysstat`
    * mpstat : CPU分析工具
    * pidstat : 进程性能分析工具
```shell
➜  ~ sudo proxychains apt-get install stress sysstat
```
    
* 几个工具

```shell
stress  --cpu percentage  --timeout seconds  # 压满cpu多少秒
        --io  percentage                     # 压满io多少秒
          
watch -d uptime  # watch: 每两秒执行一次后面的命令,-d:两次结果改变的数值高亮显示
                 # uptime: 查看平均负载. watch uptime相当于自动每2秒查看一次平均负载

mpstat -P ALL 5  # 每五秒查看所有cpu核的占用情况

pidstat -u 5     # 每5秒查看进程的cpu,io等使用情况
```

### 三种情景的模拟
#### 场景一：CPU 密集型进程
1. 首先，我们在第一个终端运行 stress 命令，模拟一个 CPU 使用率 100% 的场景：

```shell
$ stress --cpu 1 --timeout 600
接着，在第二个终端运行 uptime 查看平均负载的变化情况：

# -d 参数表示高亮显示变化的区域
$ watch -d uptime
...,  load average: 1.00, 0.75, 0.39
```
2. 最后，在第三个终端运行 mpstat 查看 CPU 使用率的变化情况：

```shell
# -P ALL 表示监控所有 CPU，后面数字 5 表示间隔 5 秒后输出一组数据
$ mpstat -P ALL 5
Linux 4.15.0 (ubuntu) 09/22/18 _x86_64_ (2 CPU)
13:30:06     CPU    %usr   %nice    %sys %iowait    %irq   %soft  %steal  %guest  %gnice   %idle
13:30:11     all   50.05    0.00    0.00    0.00    0.00    0.00    0.00    0.00    0.00   49.95
13:30:11       0    0.00    0.00    0.00    0.00    0.00    0.00    0.00    0.00    0.00  100.00
13:30:11       1  100.00    0.00    0.00    0.00    0.00    0.00    0.00    0.00    0.00    0.00
```
    从终端二中可以看到，1 分钟的平均负载会慢慢增加到 1.00，而从终端三中还可以看到，正好有一个 CPU 的使用率为 100%，但它的 iowait只有 0。这说明，平均负载的升高正是由于 CPU 使用率为 100% 。
3. 那么，到底是哪个进程导致了 CPU 使用率为 100% 呢？你可以使用 pidstat 来查询：

```shell
# 间隔 5 秒后输出一组数据
$ pidstat -u 5 1
13:37:07      UID       PID    %usr %system  %guest   %wait    %CPU   CPU  Command
13:37:12        0      2962  100.00    0.00    0.00    0.00  100.00     1  stress
```
    从这里可以明显看到，stress 进程的 CPU 使用率为 100%。

#### 场景二：I/O 密集型进程
1. 首先还是运行 stress 命令，但这次模拟 I/O 压力，即不停地执行 sync：

```shell
$ stress -i 1 --timeout 600
还是在第二个终端运行 uptime 查看平均负载的变化情况：

$ watch -d uptime
...,  load average: 1.06, 0.58, 0.37
```
2. 然后，第三个终端运行 mpstat 查看 CPU 使用率的变化情况：

```shell

# 显示所有 CPU 的指标，并在间隔 5 秒输出一组数据
$ mpstat -P ALL 5 1
Linux 4.15.0 (ubuntu)     09/22/18     _x86_64_    (2 CPU)
13:41:28     CPU    %usr   %nice    %sys %iowait    %irq   %soft  %steal  %guest  %gnice   %idle
13:41:33     all    0.21    0.00   12.07   32.67    0.00    0.21    0.00    0.00    0.00   54.84
13:41:33       0    0.43    0.00   23.87   67.53    0.00    0.43    0.00    0.00    0.00    7.74
13:41:33       1    0.00    0.00    0.81    0.20    0.00    0.00    0.00    0.00    0.00   98.99
```
    从这里可以看到，1 分钟的平均负载会慢慢增加到 1.06，其中一个 CPU 的系统 CPU 使用率升高到了 23.87，而 iowait 高达 67.53%。这说明，平均负载的升高是由于 iowait 的升高。
3. 那么到底是哪个进程，导致 iowait 这么高呢？我们还是用 pidstat 来查询：

```shell
# 间隔 5 秒后输出一组数据，-u 表示 CPU 指标
$ pidstat -u 5 1
Linux 4.15.0 (ubuntu)     09/22/18     _x86_64_    (2 CPU)
13:42:08      UID       PID    %usr %system  %guest   %wait    %CPU   CPU  Command
13:42:13        0       104    0.00    3.39    0.00    0.00    3.39     1  kworker/1:1H
13:42:13        0       109    0.00    0.40    0.00    0.00    0.40     0  kworker/0:1H
13:42:13        0      2997    2.00   35.53    0.00    3.99   37.52     1  stress
13:42:13        0      3057    0.00    0.40    0.00    0.00    0.40     0  pidstat
```
    可以发现，还是 stress 进程导致的。

#### 场景三：大量进程的场景
1. 当系统中运行进程超出 CPU 运行能力时，就会出现等待 CPU 的进程。  
比如，我们还是使用 stress，但这次模拟的是 8 个进程：

```shell
$ stress -c 8 --timeout 600
由于系统只有 2 个 CPU，明显比 8 个进程要少得多，因而，系统的 CPU 处于严重过载状态，平均负载高达 7.97：

$ uptime
...,  load average: 7.97, 5.93, 3.02
```
2. 接着再运行 pidstat 来看一下进程的情况：

```shell
# 间隔 5 秒后输出一组数据
$ pidstat -u 5 1
14:23:25      UID       PID    %usr %system  %guest   %wait    %CPU   CPU  Command
14:23:30        0      3190   25.00    0.00    0.00   74.80   25.00     0  stress
14:23:30        0      3191   25.00    0.00    0.00   75.20   25.00     0  stress
14:23:30        0      3192   25.00    0.00    0.00   74.80   25.00     1  stress
14:23:30        0      3193   25.00    0.00    0.00   75.00   25.00     1  stress
14:23:30        0      3194   24.80    0.00    0.00   74.60   24.80     0  stress
14:23:30        0      3195   24.80    0.00    0.00   75.00   24.80     0  stress
14:23:30        0      3196   24.80    0.00    0.00   74.60   24.80     1  stress
14:23:30        0      3197   24.80    0.00    0.00   74.80   24.80     1  stress
14:23:30        0      3200    0.00    0.20    0.00    0.20    0.20     0  pidstat
```
    可以看出，8 个进程在争抢 2 个 CPU，每个进程等待 CPU 的时间（也就是代码块中的 %wait 列）高达 75%。这些超出 CPU 计算能力的进程，最终导致 CPU 过载。

